In [2]:
import nltk
import os
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

with open("BooksCleaned/Book46.txt") as book:
    contents = book.read()
    #print(contents)
    tokens = nltk.word_tokenize(contents)
    #print(tokens)
    parts_of_speech = nltk.pos_tag(tokens)
    #print(parts_of_speech)


[nltk_data] Downloading package punkt to
[nltk_data]     /home/malvinaclavering/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/malvinaclavering/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
import nltk
from nltk.corpus import wordnet   #Import wordnet from the NLTK
def find_syn(keyword):    
    syn = list()
    for synset in wordnet.synsets(keyword):
        for lemma in synset.lemmas():
            syn.append(lemma.name())    #add the synonyms
    #print('Synonyms: ' + str(syn))
    return str(syn)

In [32]:
def find_occurances_of_keyword(keyword,text):
    list_of_indexes=[]
    for index,word in enumerate(text):
        if keyword == word:
            list_of_indexes.append(index)
    return list_of_indexes

In [6]:
def find_adjectives(indexes, parts_of_speech_list):
    list_of_adjectives=[]
    for index in indexes:
        if parts_of_speech_list[index-1][1]=="JJ" or \
        parts_of_speech_list[index-1][1]=="JJR" \
        or parts_of_speech_list[index-1][1]=="JJS":
            list_of_adjectives.append(parts_of_speech_list[index-1][0])
    return list_of_adjectives

In [7]:
def find_words_that_describe_keyword(keyword,path):
    with open(path) as book:
        contents = book.read()
        tokens = nltk.word_tokenize(contents)
        parts_of_speech = nltk.pos_tag(tokens)
    keyword_indexes=find_occurances_of_keyword(keyword,tokens)
    adjectives=find_adjectives(keyword_indexes,parts_of_speech)
    return adjectives
find_words_that_describe_keyword("gentleman", "BooksCleaned/Book922.txt")

['old', 'old', 'old']

In [54]:
def find_words_in_all_books_that_describe_keyword(keyword):
    adj=[]
    for book in os.listdir("BooksCleaned"):
        adj+=(find_words_that_describe_keyword(keyword,f'BooksCleaned/{book}')) 
    return(adj)
adj_lawyer=find_words_in_all_books_that_describe_keyword("lawyer")

In [33]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
#def sentiment_countifier(adjectives):
#    sia = SentimentIntensityAnalyzer()
#    str1 = " " 
#    words =str1.join(adjectives)    
#    print(words)
#    sentiments = sia.polarity_scores(words)
#    return sentiments
#print(sentiment_countifier(adj_lawyer))
def sentiment_countifier_individual(adjectives):
    sia = SentimentIntensityAnalyzer()
    pos_scores=[]    
    neu_scores=[]    
    neg_scores=[] 
    for word in adjectives:    
        sentiments = sia.polarity_scores(word)
        print(sentiments)
        pos_scores.append(sentiments["pos"])
        neu_scores.append(sentiments["neu"])
        neg_scores.append(sentiments["neg"])
    pos_score= sum(pos_scores) / len(pos_scores)
    neu_score= sum(neu_scores) / len(neu_scores)
    neg_score= sum(neg_scores) / len(neg_scores)    
    return [pos_score,neu_score,neg_score]
print(sentiment_countifier_individual(adj_lawyer))


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/malvinaclavering/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


NameError: name 'adj_lawyer' is not defined

In [38]:
def find_sentences_with_keyword(keyword, text):
    """
    Find the sentence that contextualizes the occurence of a keyword and return a list
    of indices that represent the range of each sentence containing the keyword.
    
    Args:
        keyword: a string representing a word to contextualize in a sentence.
        text: a tokenized text given as a list of strings of words and punctuation.
        
    Returns:
        A list of tuples representing the ranges of each sentence containing the keyword."""
    keyword_locations = find_occurances_of_keyword(keyword, text)
    end_punctuation = [".", "!", "?"]
    sentence_ranges = []
    for i in keyword_locations:
        next_word = i + 1
        while text[next_word] not in end_punctuation:
            next_word += 1
        end_location = next_word + 1
        previous_word = i-1
        while text[previous_word] not in end_punctuation:
            previous_word -= 1
        start_location = previous_word + 1
        sentence_ranges.append((start_location, end_location))
    return sentence_ranges

find_sentences_with_keyword("gentleman", tokens)


[(435, 511),
 (2508, 2533),
 (2567, 2617),
 (2649, 2665),
 (2687, 2704),
 (2770, 2819),
 (2988, 3001),
 (3444, 3494),
 (11675, 11752),
 (12041, 12091),
 (26584, 26609),
 (26701, 26725),
 (31263, 31286),
 (34164, 34203),
 (34203, 34240),
 (34240, 34263),
 (34348, 34361),
 (34466, 34472)]